# 라이브러리 로드

In [12]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

# 모델 생성
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report


# 가상고객 데이터 생성
import random
pd.options.display.float_format = '{:.5f}'.format


%matplotlib inline

# 모델 생성

In [65]:
def churn_prediction(df, test_data):
    # 범주형 컬럼 One-Hot Encoding
    encoding_df = pd.get_dummies(df, columns=['Membership', 'Contract'])

    # 학습/테스트셋 분리
    y_target = encoding_df['Churn Value']
    X_data = encoding_df.drop(['Churn Value'], axis=1, inplace=False)

    X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=156)

    # 학습 데이터셋(수치형 컬럼) 정규화
    ## Initialize variable
    X_train_origin, X_test_origin, y_train_origin, y_test_origin = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()

    ## Numeric Only
    Numeric_column_list = []
    for i in range(len(X_data.columns)):
        if X_data[X_data.columns[i]].dtype == 'float64' or X_data[X_data.columns[i]].dtype == 'int64':
            Numeric_column_list.append(X_data.columns[i])

    numeric_train_data, numeric_test_data = X_train[Numeric_column_list], X_test[Numeric_column_list]

    scaler = MinMaxScaler()

    X_train[Numeric_column_list] = scaler.fit_transform(numeric_train_data)
    X_test[Numeric_column_list] = scaler.transform(numeric_test_data)

    # 선정한 모델로 학습
    best_model = SGDClassifier(random_state=42, alpha=0.001, loss='modified_huber',
                               max_iter=100, penalty='l1', tol=1e-05)

    best_model.fit(X_train, y_train)

    #y_pred = best_model.predict(X_test)

    # 가상 데이터로 예측 확률 계산(test_data)
    
    ## 범주형 컬럼 One-Hot Encoding
    test_data = pd.get_dummies(test_data, columns=['Membership', 'Contract'])
    
    ## 정규화 (Numeric Only)
    test_Numeric_column_list = []
    for i in range(len(test_data.columns)):
        if test_data[test_data.columns[i]].dtype == 'float64' or test_data[test_data.columns[i]].dtype == 'int64' or test_data[test_data.columns[i]].dtype == 'int64':
            test_Numeric_column_list.append(test_data.columns[i])
            
    test_data[test_Numeric_column_list] = scaler.transform(test_data[test_Numeric_column_list].values)  # .values 추가
    
    print(test_data)
    
    # 최적의 모델을 사용하여 테스트 데이터의 클래스 확률을 예측합니다.
    predicted_proba = best_model.predict_proba(test_data)

    # 각 테스트 데이터에 대한 클래스 1일 확률을 출력합니다.
    for i, data in enumerate(test_data, start=1):
        predicted_proba = best_model.predict_proba([data])
        probability_of_class_1 = predicted_proba[0][1]
        print(f"테스트 데이터 {i}의 가상 고객이 이탈할 확률:", probability_of_class_1)
        print(f"테스트 데이터 {i}의 가상 고객 이탈 점수:", int(probability_of_class_1 * 100), "점")

In [115]:
def churn_prediction(df, test_data):
    # 범주형 컬럼 One-Hot Encoding
    encoding_df = pd.get_dummies(df, columns=['Membership', 'Contract'])

    # 학습/테스트셋 분리
    y_target = encoding_df['Churn Value']
    X_data = encoding_df.drop(['Churn Value'], axis=1, inplace=False)

    X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=156)

    # 학습 데이터셋(수치형 컬럼) 정규화
    ## Initialize variable
    X_train_origin, X_test_origin, y_train_origin, y_test_origin = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()

    ## Numeric Only
    Numeric_column_list = []
    for i in range(len(X_data.columns)):
        if X_data[X_data.columns[i]].dtype == 'float64' or X_data[X_data.columns[i]].dtype == 'int64':
            Numeric_column_list.append(X_data.columns[i])

    numeric_train_data, numeric_test_data = X_train[Numeric_column_list], X_test[Numeric_column_list]

    scaler = MinMaxScaler()

    X_train[Numeric_column_list] = scaler.fit_transform(numeric_train_data)
    X_test[Numeric_column_list] = scaler.transform(numeric_test_data)

    # 선정한 모델로 학습
    best_model = SGDClassifier(random_state=42, alpha=0.001, loss='modified_huber',
                               max_iter=100, penalty='l1', tol=1e-05)

    best_model.fit(X_train, y_train)

    #y_pred = best_model.predict(X_test)

    # 가상 데이터로 예측 확률 계산(test_data)
    
    ## 범주형 컬럼 One-Hot Encoding
    encoding_test_data = pd.get_dummies(test_data, columns=['Membership', 'Contract'])
    
    ## 정규화 (Numeric Only)
    test_Numeric_column_list = []
    for i in range(len(test_data.columns)):
        if test_data[test_data.columns[i]].dtype != 'object':
            test_Numeric_column_list.append(test_data.columns[i])
            
    numeric_test_data = test_data[test_Numeric_column_list]

    scaler = MinMaxScaler()
    
    numeric_test_data[test_Numeric_column_list] = scaler.fit_transform(numeric_test_data)
            
    # 각 테스트 데이터에 대한 클래스 1일 확률을 출력합니다.
    for i, data in enumerate(numeric_test_data, start=1):
        predicted_proba = best_model.predict_proba([data])
        probability_of_class_1 = predicted_proba[0][1]
        
    # 결과를 데이터프레임에 추가
    test_data['이탈점수'] = predicted_probabilities[:, 1]*100  # 클래스 1의 확률을 선택
    
    return test_data  # test_data 반환

# 학습에 이용할 데이터 로드

In [116]:
df = pd.read_excel("Churn.xlsx")
df.head()

,Age,Number of Dependents,Membership,Satisfaction Score,Tech services,Streaming services,Combined Product,Contract,Tenure in Months,Monthly Charge,Total Revenue,Churn Value
0,78,0,None,3,1,1,2,Month-to-Month,1,39.65000,59.65000,1
1,74,1,Offer E,3,1,0,1,Month-to-Month,8,80.65000,1024.10000,1
2,71,3,Offer D,2,0,2,1,Month-to-Month,18,95.45000,1910.88000,1
3,78,1,Offer C,2,2,2,1,Month-to-Month,25,98.50000,2995.07000,1
4,80,1,Offer C,2,0,0,1,Month-to-Month,37,76.50000,3102.36000,1


# 가상고객데이터 생성

In [117]:
# 빈 데이터프레임 생성
test_data = pd.DataFrame()

test_data['Age'] = np.random.randint(19, 81, 100)
test_data['Number of Dependents'] = np.random.randint(0, 6, 100)
test_data['Membership'] = np.random.choice(['None', 'Offer A', 'Offer B', 'Offer C', 'Offer D', 'Offer E'], size=100)
test_data['Satisfaction Score'] = np.random.randint(1, 6, 100)
test_data['Tech services'] = np.random.randint(0, 5, 100)
test_data['Streaming services'] = np.random.randint(0, 3, 100)
test_data['Combined Product'] = np.random.randint(1, 5, 100)
test_data['Contract'] = np.random.choice(['Month-to-Month', 'One Year', 'Two Year'], size=100)
test_data['Tenure in Months'] = np.random.randint(1, 73, 100)
test_data['Monthly Charge'] = np.random.randint(18, 119, 100)

# 'Total Revenue' 컬럼 생성
random_values = np.random.randint(-50, 151, size=100)
revenues = (test_data['Tenure in Months'] * test_data['Monthly Charge']) - random_values
test_data['Total Revenue'] = np.maximum(revenues, 0)

test_data

,Age,Number of Dependents,Membership,Satisfaction Score,Tech services,Streaming services,Combined Product,Contract,Tenure in Months,Monthly Charge,Total Revenue
0,54,5,Offer E,5,3,2,1,One Year,4,115,326
1,24,3,None,1,0,2,2,One Year,37,89,3259
2,20,3,Offer D,2,1,2,4,One Year,72,90,6378
3,60,1,Offer D,2,3,1,3,Two Year,64,53,3389
4,39,1,None,1,0,1,3,Two Year,42,78,3301
...,...,...,...,...,...,...,...,...,...,...,...
95,21,2,Offer A,5,1,2,3,Month-to-Month,15,97,1500
96,77,4,Offer C,4,2,2,2,One Year,17,39,582
97,77,3,Offer E,2,0,0,4,Two Year,32,73,2343
98,72,4,Offer E,2,0,1,3,One Year,46,82,3658


# 함수 호출

In [118]:
result = churn_prediction(df, test_data)
print(result)  # test_data를 출력

ValueError: Expected 2D array, got 1D array instead:
array=['Age'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [119]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Age                   100 non-null    int32 
 1   Number of Dependents  100 non-null    int32 
 2   Membership            100 non-null    object
 3   Satisfaction Score    100 non-null    int32 
 4   Tech services         100 non-null    int32 
 5   Streaming services    100 non-null    int32 
 6   Combined Product      100 non-null    int32 
 7   Contract              100 non-null    object
 8   Tenure in Months      100 non-null    int32 
 9   Monthly Charge        100 non-null    int32 
 10  Total Revenue         100 non-null    int32 
dtypes: int32(9), object(2)
memory usage: 5.2+ KB
